In [31]:
# Importação das bibliotecas usadas no desenvolvimento do projeto
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SequentialFeatureSelector
from statistics import mean, median

In [32]:
# Importação dos conjutos de amostras
dados_treino = pd.read_csv("conjunto_de_treinamento.csv", delimiter=",", decimal=".")
dados_teste = pd.read_csv("conjunto_de_teste.csv", delimiter=",", decimal=".")

In [33]:
dados_teste.head()

,id_solicitante,produto_solicitado,dia_vencimento,forma_envio_solicitacao,tipo_endereco,sexo,idade,estado_civil,qtde_dependentes,grau_instrucao,...,estado_onde_trabalha,possui_telefone_trabalho,codigo_area_telefone_trabalho,meses_no_trabalho,profissao,ocupacao,profissao_companheiro,grau_instrucao_companheiro,local_onde_reside,local_onde_trabalha
0,20001,1,25,presencial,1,M,37,2,0,0,...,,N,,0,0.0,0.0,0.0,0.0,384.0,384.0
1,20002,1,10,internet,1,F,31,2,0,0,...,RJ,N,,0,9.0,5.0,NaN,NaN,275.0,275.0
2,20003,1,10,internet,1,F,18,2,0,0,...,RS,N,,0,9.0,2.0,NaN,NaN,948.0,948.0
3,20004,1,10,presencial,1,F,55,2,0,0,...,,N,,0,9.0,1.0,0.0,0.0,581.0,581.0
4,20005,1,10,presencial,1,F,55,1,0,0,...,,N,,0,0.0,1.0,0.0,0.0,573.0,573.0


In [34]:
dados_treino.head()

,id_solicitante,produto_solicitado,dia_vencimento,forma_envio_solicitacao,tipo_endereco,sexo,idade,estado_civil,qtde_dependentes,grau_instrucao,...,possui_telefone_trabalho,codigo_area_telefone_trabalho,meses_no_trabalho,profissao,ocupacao,profissao_companheiro,grau_instrucao_companheiro,local_onde_reside,local_onde_trabalha,inadimplente
0,1,1,10,presencial,1,M,85,2,0,0,...,N,,0,9.0,1.0,0.0,0.0,600.0,600.0,0
1,2,1,25,internet,1,F,38,1,0,0,...,N,,0,2.0,5.0,NaN,NaN,492.0,492.0,0
2,3,1,20,internet,1,F,37,2,0,0,...,N,,0,NaN,NaN,NaN,NaN,450.0,450.0,1
3,4,1,20,internet,1,M,37,1,1,0,...,Y,54,0,9.0,2.0,NaN,NaN,932.0,932.0,1
4,5,7,1,internet,1,F,51,1,3,0,...,N,,0,9.0,5.0,NaN,NaN,440.0,440.0,1


# Verificação de Outliers

Após gerar os gráficos de todas as colunas do datasets, percebeu-se que existe um outlier que possui 53 dependentes cadastrados. As demais variáveis não apresentam outliers.

In [35]:
#for coluna in dados_treino.columns:
#     dados_treino.plot.scatter(x=coluna, y='inadimplente')

dados_treino[dados_treino['qtde_dependentes'] > 15] #printa o outlier
dados_treino = dados_treino.drop(8489)

# Tratamento de dados

## Lidando com os NaN

Como é possível observar nas 5 primeiras linhas dos dados de treino e teste, existem dados faltantes "NaN". Dado isto, quantifica-se a perda de dados ao simplesmente eliminar as linhas com NaN.

In [36]:
linha_treino_com_na = dados_treino.shape[0]
linha_treino_sem_na = dados_treino.dropna().shape[0]

print("Perda de %0.2f%% dos dados de treino ao retirar linhas com informações faltantes." %(100 - linha_treino_sem_na*100/linha_treino_com_na))

Perda de 65.12% dos dados de treino ao retirar linhas com informações faltantes.


A perda de 65.12% do dataset é algo péssimo para o modelo pois perde grande parte da representação do universo. Para prosseguir sem perder essas amostras, optou-se por substituir os NaN de forma a manter a característica das distribuições.

Para fazer essas substituições, primeiro faz-se o tratamento dos dados nos datasets de treino e teste. A seguir seguem as modificaçãoes feitas sobre os dados.

## Conversão dos dados e remoção de colunas

Primeiramente faz-se a análise do arquivo "dicionario_de_dados.xlsx" e segue-se as recomendações para remover as colunas com erro de preenchimento e colunas que visivelmente agregam pouco valor (por ter alta correlação com outra coluna. Ex. colunas de estados onde trabalha, habita e nasceu).

In [37]:
# forma_envio_solicitacao -> Abrir em 3 colunas (internet, correio e presencial)
# sexo -> Abrir em 3 colunas (masculino, feminino, nao informado). Se vazio colocar N
# grau_instrucao -> Remover
# estado_onde_nasceu -> Remover, possui alta correlação  com "estado_onde_reside"
# estado_onde_reside -> Transformar em regiões do país
# possui_telefone_residencial -> Transformar em 1 - Sim 0 - Nao
# possui_telefone_celular ->  Remover
# qtde_contas_bancarias_especiais -> Remover
# vinculo_formal_com_empresa -> Transformar em 1 - Sim 0 - Nao
# estado_onde_trabalha -> Inútil, apenas 0.2% está preenchido
# possui_telefone_trabalho -> Transformar em 1 - Sim 0 - Nao
# meses_no_trabalho -> Remover

# deleta as colunas desnecessárias/com dados corrompidos
# codigo_area_telefone_residencial, codigo_area_telefone_trabalho, estado_onde_trabalha, estado_onde_nasceu
dados_treino = dados_treino.drop(columns=['id_solicitante', 'grau_instrucao', 'possui_telefone_celular',\
                                          'qtde_contas_bancarias_especiais', 'meses_no_trabalho'])
dados_teste = dados_teste.drop(columns=['grau_instrucao', 'possui_telefone_celular',\
                                          'qtde_contas_bancarias_especiais', 'meses_no_trabalho'])

In [38]:
# splita as colunas na lista "valores_a_trocar" em uma coluna para cada opção dentre as possíveis
# caso o valor de uma linha da coluna sexo seja NaN, troca por N - Não informado
valores = {'sexo':'N'}

valores_a_trocar = ['possui_telefone_residencial', 'vinculo_formal_com_empresa', 'possui_telefone_trabalho', 'forma_envio_solicitacao', 'sexo']

dados_treino = dados_treino.fillna(value=valores)
dados_teste = dados_teste.fillna(value=valores)

dados_treino = pd.get_dummies(dados_treino, columns=valores_a_trocar)
dados_teste = pd.get_dummies(dados_teste, columns=valores_a_trocar)

# drop necessário para tirar uma coluna de bug
dados_treino = dados_treino.drop(columns=['sexo_ '])
dados_teste = dados_teste.drop(columns=['sexo_ '])

In [39]:
# Troca os estados das colunas na lista abaixo pela região do estado e faz o get_dummie da coluna
valores_a_trocar = ['estado_onde_reside', 'estado_onde_nasceu', 'estado_onde_trabalha']

dicionario = {'AC':'NO', 'AP':'NO', 'AM':'NO', 'PA':'NO', 'RO':'NO', 'RR':'NO', 'TO':'NO',\
              'MA':'NE', 'PI':'NE', 'CE':'NE', 'RN':'NE', 'PB':'NE', 'PE':'NE', \
             'AL':'NE', 'SE':'NE', 'BA':'NE', 'MT':'CO', 'MS':'CO', 'GO':'CO',\
             'DF':'CO', 'RJ':'SD', 'MG':'SD', 'SP':'SD', 'ES':'SD', 'SC':'SU', 'PR':'SU', 'RS':'SU', 'XX':'NC'}

for coluna in valores_a_trocar:
    dados_treino[coluna] = dados_treino[coluna].map(dicionario, na_action='NC')
    dados_teste[coluna] = dados_teste[coluna].map(dicionario, na_action='NC')
    
dados_treino = pd.get_dummies(dados_treino, columns=valores_a_trocar)
dados_teste = pd.get_dummies(dados_teste, columns=valores_a_trocar)

In [40]:
# troca os valores ' ' das colunas listadas por NaN
valores_a_trocar = ['codigo_area_telefone_residencial', 'codigo_area_telefone_trabalho']

for coluna in valores_a_trocar:        
    dados_treino[coluna] = dados_treino[coluna].replace(' ', np.nan)
    dados_teste[coluna] = dados_teste[coluna].replace(' ', np.nan)

Após as tratativas nas últimas 4 células, finalmente tem-se dados com todas as linhas preenchidas. Ao longo das tratativas alguns dados foram transformados e/ou removidos, alterando de alguma forma na precisão do modelo.

Agora, com todos os dados transformados pode-se substituir os valores faltantes por alguma métrica.

## Substituindo os dados faltantes

A fim de simplificar as operações, substitui-se os dados faltantes das colunas não quantitativas pela mediana e os das colunas quantitativas pela média.

In [41]:
dados_treino = dados_treino.astype(float)
dados_teste = dados_teste.astype(float)

In [42]:
# Substitui os dados NaN pela mediana ou média da coluna

colunas_quantitativas = ['idade', 'meses_na_residencia']
# as variáveis de renda não entram nessa lista pois a média colocaria um viés na realidade do universo das amostras

for coluna in dados_treino.columns:
    if coluna == 'inadimplente':
        continue
    elif coluna in colunas_quantitativas:
        dados_teste[[coluna]] = dados_teste[[coluna]].fillna(dados_teste.mean())
        dados_treino[[coluna]] = dados_treino[[coluna]].fillna(dados_treino.mean())
    else:
        dados_teste[[coluna]] = dados_teste[[coluna]].fillna(dados_teste.median())
        dados_treino[[coluna]] = dados_treino[[coluna]].fillna(dados_treino.median())

# Criando os modelos classificadores

Neste ponto os dados estão todos em valor numérico com todas as colunas preenchidas. Pode-se, portanto, prosseguir para a criação do modelo preditivo.

## Escolha de variáveis

In [43]:
# calcula o coeficiente de Pearson para cada coluna em relação ao alvo
pearson_coef = {coluna:round(abs(pearsonr(dados_treino[coluna], dados_treino['inadimplente'])[0]),4) for coluna in dados_treino.columns}

pearson_coef

{'produto_solicitado': 0.0301,
 'dia_vencimento': 0.0804,
 'tipo_endereco': 0.0044,
 'idade': 0.1207,
 'estado_civil': 0.0313,
 'qtde_dependentes': 0.0207,
 'nacionalidade': 0.0005,
 'codigo_area_telefone_residencial': 0.0132,
 'tipo_residencia': 0.0181,
 'meses_na_residencia': 0.0323,
 'possui_email': 0.0057,
 'renda_mensal_regular': 0.0009,
 'renda_extra': 0.0058,
 'possui_cartao_visa': 0.0013,
 'possui_cartao_mastercard': 0.02,
 'possui_cartao_diners': 0.0025,
 'possui_cartao_amex': 0.0011,
 'possui_outros_cartoes': 0.0032,
 'qtde_contas_bancarias': 0.0142,
 'valor_patrimonio_pessoal': 0.0012,
 'possui_carro': 0.0127,
 'codigo_area_telefone_trabalho': 0.0084,
 'profissao': 0.0126,
 'ocupacao': 0.0386,
 'profissao_companheiro': 0.0232,
 'grau_instrucao_companheiro': 0.0228,
 'local_onde_reside': 0.0189,
 'local_onde_trabalha': 0.0189,
 'inadimplente': 1.0,
 'possui_telefone_residencial_N': 0.0808,
 'possui_telefone_residencial_Y': 0.0808,
 'vinculo_formal_com_empresa_N': 0.0062,
 'vi

In [44]:
from operator import itemgetter

coef_ordenados = sorted(pearson_coef.items(), key=itemgetter(1))
media_coef = []

for indice, tupla in enumerate(coef_ordenados):
    coef = tupla[1]
    coluna = tupla[0]
    for colunas in dados_treino.columns:
        if colunas not in ['inadimplente', coluna]:
            media_coef.append(abs(round(pearsonr(dados_treino[coluna], dados_treino[colunas])[0],2)))
    correlacao_media = mean(media_coef)
    coef_ordenados[indice] = [coluna, coef, round(correlacao_media, 4), round(coef/correlacao_media, 4)]

In [77]:
coef_ordenados = sorted(coef_ordenados, key=itemgetter(3))
coef_ordenados

[['nacionalidade', 0.0005, 0.0656, 0.0076],
 ['renda_mensal_regular', 0.0009, 0.0367, 0.0245],
 ['estado_onde_nasceu_SD', 0.0013, 0.0413, 0.0315],
 ['possui_cartao_amex', 0.0011, 0.0299, 0.0368],
 ['sexo_N', 0.0014, 0.0371, 0.0377],
 ['valor_patrimonio_pessoal', 0.0012, 0.0256, 0.0468],
 ['possui_cartao_visa', 0.0013, 0.0267, 0.0487],
 ['possui_cartao_diners', 0.0025, 0.0344, 0.0726],
 ['possui_outros_cartoes', 0.0032, 0.0316, 0.1013],
 ['estado_onde_trabalha_NO', 0.0042, 0.0359, 0.1171],
 ['estado_onde_trabalha_SD', 0.0064, 0.0524, 0.1222],
 ['estado_onde_trabalha_SU', 0.0075, 0.0585, 0.1282],
 ['tipo_endereco', 0.0044, 0.0342, 0.1286],
 ['forma_envio_solicitacao_internet', 0.0063, 0.0486, 0.1295],
 ['estado_onde_reside_SD', 0.0074, 0.0567, 0.1306],
 ['vinculo_formal_com_empresa_Y', 0.0062, 0.0453, 0.1368],
 ['codigo_area_telefone_trabalho', 0.0084, 0.06, 0.1399],
 ['estado_onde_nasceu_NO', 0.0092, 0.0606, 0.1519],
 ['possui_email', 0.0057, 0.0374, 0.1523],
 ['vinculo_formal_com_empre

In [78]:
indice_aceitavel = 13 #30 ou 29
variaveis_escolhidas = []

for item in coef_ordenados[indice_aceitavel:]:
    variaveis_escolhidas.append(item[0])

variaveis_escolhidas

['forma_envio_solicitacao_internet',
 'estado_onde_reside_SD',
 'vinculo_formal_com_empresa_Y',
 'codigo_area_telefone_trabalho',
 'estado_onde_nasceu_NO',
 'possui_email',
 'vinculo_formal_com_empresa_N',
 'estado_onde_nasceu_CO',
 'estado_onde_reside_NO',
 'renda_extra',
 'possui_carro',
 'profissao',
 'codigo_area_telefone_residencial',
 'forma_envio_solicitacao_presencial',
 'qtde_contas_bancarias',
 'estado_onde_trabalha_CO',
 'possui_telefone_trabalho_Y',
 'estado_onde_reside_CO',
 'possui_telefone_trabalho_N',
 'forma_envio_solicitacao_correio',
 'estado_onde_nasceu_NE',
 'tipo_residencia',
 'local_onde_trabalha',
 'local_onde_reside',
 'possui_cartao_mastercard',
 'qtde_dependentes',
 'grau_instrucao_companheiro',
 'profissao_companheiro',
 'sexo_M',
 'sexo_F',
 'produto_solicitado',
 'estado_civil',
 'estado_onde_reside_NE',
 'meses_na_residencia',
 'estado_onde_trabalha_NE',
 'ocupacao',
 'estado_onde_nasceu_SU',
 'estado_onde_reside_SU',
 'dia_vencimento',
 'possui_telefone_

In [47]:
dados_treino = dados_treino[variaveis_escolhidas]
dados_teste = dados_teste[variaveis_escolhidas[:-1]]

## Separação dos dados

In [59]:
#separacao do conjunto de treino em alvo e features e em subconjunto de teste e treino
dados_treino = dados_treino.sample(frac=1, random_state=135)

X = dados_treino.drop(columns=['inadimplente'])
Y = dados_treino['inadimplente']

teste_sample = 0.85

x_treino = X[:int(len(X)*teste_sample)]
y_treino = Y[:int(len(X)*teste_sample)]

x_teste = X[int(len(X)*teste_sample):]
y_teste = Y[int(len(X)*teste_sample):]

## Scaling dos dados

In [49]:
standard_scaler =  StandardScaler() #MinMaxScaler()
X = standard_scaler.fit_transform(X)

## Criando KNN

In [68]:
# Ajuste grosso de hiperparametro

print("  K   |   TESTE   |  TREINO  ")
for numero_kneighbors in range(90, 170, 15):
    classificadorKNN = KNeighborsClassifier(n_neighbors=numero_kneighbors, weights='uniform')
    classificadorKNN = classificadorKNN.fit(x_treino, y_treino)

    predicao_teste = classificadorKNN.predict(x_teste)
    predicao_treino = classificadorKNN.predict(x_treino)
    
    print("%d    " %numero_kneighbors, end='')
    print(f"  {round(sum(predicao_teste == y_teste)*100/len(y_teste),2)}%      {round(100*sum(predicao_treino == y_treino)/len(y_treino), 2)}%")

  K   |   TESTE   |  TREINO  
90      56.63%      59.33%
105      57.2%      59.02%
120      56.77%      58.72%
135      56.27%      58.66%
150      56.9%      58.41%
165      57.3%      58.52%


In [72]:
# Verifica a acurácia do melhor classificador KNN

classificadorKNN = KNeighborsClassifier(n_neighbors=165, weights='uniform') #151

scores = cross_val_score(classificadorKNN, X, Y, cv=3)
print("%.4f" %mean(scores))

0.5679


## Criando SVM com Kernel

In [70]:
# Ajuste de hiperparametro

print("  C   |   TESTE   |  TREINO  ")
for c in np.arange(0.01, 0.15, 0.05):
    classificadorSVM = SVC(kernel='rbf', C=c, gamma='auto')
    
    classificadorSVM = classificadorSVM.fit(x_treino, y_treino)

    predicao_teste = classificadorSVM.predict(x_teste)
    predicao_treino = classificadorSVM.predict(x_treino)

    print("%.2f" %c, end='')
    print(f"  {round(sum(predicao_teste == y_teste)*100/len(y_teste),2)}%      {round(100*sum(predicao_treino == y_treino)/len(y_treino), 2)}%")

  C   |   TESTE   |  TREINO  
0.01  49.87%      50.02%
0.06  49.87%      50.02%
0.11  49.87%      50.53%


In [71]:
# Verifica a acurácia do melhor classificador SVM dentro e fora da amostra de teste

classificadorSVM = SVC(kernel='rbf', C=0.08, gamma='auto')

scores = cross_val_score(classificadorSVM, X, Y, cv=3)
print("%0.2f   " %0.08, end='')
print("%.4f" %mean(scores)) 

0.08   0.5054


## Criando Logistic Regression (talvez trocar por um RF msm)

In [62]:
# Ajuste de hiperparametro

print("  C   |   %  ")
for expo in np.arange(-3, 4, 0.5):
    C = 10**expo
    classificadorLRC = LogisticRegression(penalty='l2', C=C, max_iter=10000)
    
    scores = cross_val_score(classificadorLRC, X, Y, cv=3)
    print("%0.3f    " %C, end='')
    print("%.4f" %mean(scores))

  C   |   %  
0.001    0.5745
0.003    0.5814
0.010    0.5843
0.032    0.5839
0.100    0.5839
0.316    0.5842
1.000    0.5839
3.162    0.5852
10.000    0.5846
31.623    0.5830
100.000    0.5841
316.228    0.5842
1000.000    0.5830
3162.278    0.5847


In [67]:
# Verifica a acurácia do melhor classificador SVM dentro e fora da amostra de teste
classificadorLRC = LogisticRegression(penalty='l2', C=0.316, max_iter=10000)
classificadorLRC = classificadorLRC.fit(x_treino, y_treino)

predicao_teste = classificadorLRC.predict(x_teste)
predicao_treino = classificadorLRC.predict(x_treino)

print("TESTE   TREINO")
print(f"{round(sum(predicao_teste == y_teste)/len(y_teste),4)}   {round(sum(predicao_treino == y_treino)/len(y_treino), 4)}")

TESTE   TREINO
0.5867   0.587


## Criando o One vs. All

In [73]:
class OneVsAll():
    def __init__(self):
        #self.KNN = KNeighborsClassifier(n_neighbors=319, weights='uniform')
        self.SVM = SVC(kernel='rbf', C=0.08)
        #self.LR = LogisticRegression(penalty='l2', C=0.002, max_iter=10000)
    def fit_trinca(self, X, Y):
        #self.KNN = self.KNN.fit(X, Y)
        self.SVM = self.SVM.fit(X, Y)
        #self.LR = self.LR.fit(X, Y)
    def predict_trinca(self, X):
        y_predict_KNN = self.SVM.predict(X) #self.KNN.predict(X)
        y_predict_SVM = self.SVM.predict(X)
        y_predict_LRC = self.SVM.predict(X) #self.LR.predict(X)

        return [median([y_predict_KNN[indice], y_predict_SVM[indice], y_predict_LRC[indice]]) for indice in range(len(y_predict_KNN))]

In [74]:
classificador = OneVsAll()

# Avaliação do modelo

In [75]:
## matriz de conf, acuracia e etc
classificador.fit_trinca(x_treino, y_treino)
y_one_vs_all_teste = classificador.predict_trinca(x_teste)
y_one_vs_all_treino = classificador.predict_trinca(x_treino)

In [76]:
print("TESTE TREINO")
print(round(sum(y_one_vs_all_teste == y_teste)*100/len(y_teste), 2), round(sum(y_one_vs_all_treino == y_treino)*100/len(y_treino), 2))

TESTE TREINO
51.23 52.07


# Geração do arquivo de respostas

In [79]:
x_treino_final = dados_treino.drop(columns=['inadimplente'])
y_treino_final = dados_treino['inadimplente']

x_resposta = dados_teste

x_treino_final = standard_scaler.fit_transform(x_treino_final)
x_resposta = standard_scaler.fit_transform(x_resposta)

classificador.fit_trinca(x_treino_final, y_treino_final)
y_resposta = classificador.predict_trinca(x_resposta)

In [80]:
id_solicitante = [x+20001 for x in range(5000)]

dados_resposta = pd.DataFrame(list(zip(id_solicitante, y_resposta)), columns=['id_solicitante', 'inadimplente'])

In [81]:
dados_resposta.to_csv("arquivo_resposta.csv", index=False)